<a href="https://colab.research.google.com/github/VamsiMadhav10/Recommendation_system/blob/main/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163479 sha256=ba4eb297764f4741070f34a9e43c816aaf339f7c50dbe7e4e5a653e6ac8b972b
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


# Importing Libraries

In [ ]:
import pandas as pd

import ast

from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns

import numpy as np

import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors

from scipy.sparse import csr_matrix

from surprise import Reader, Dataset

from surprise.model_selection import train_test_split

from surprise import SVDpp, accuracy

from surprise.model_selection import cross_validate

from collections import defaultdict

from surprise import SVD, SVDpp, NMF

from surprise import SlopeOne, CoClustering

import matplotlib

import time

In [ ]:
df_movies = pd.read_csv('/content/movies.csv')

df_ratings = pd.read_csv('/content/ratings.csv')

# Graph According to the rating

In [ ]:
df.head(5)

# Considering only the Movies which has atleast 50 Ratings

In [ ]:
min_ratings = 50

print("Number of movies which have more than {} ratings: {}".format(min_ratings, len(df[df[('userId', 'count')]>=min_ratings])))

Number of movies which have more than 50 ratings: 1060


In [ ]:
df_top_movies = df[df[('userId', 'count')]>=min_ratings]

df_ratings_with_top_movies = df_ratings[df_ratings['movieId'].isin(list(df_top_movies.index))]

In [ ]:
min_movies_rated = 100

df_users = df_ratings_with_top_movies[['movieId','userId']].groupby(['userId']).agg(['count']).sort_values(('movieId','count'),ascending=False)

df_top_rating_users = df_users[df_users[('movieId', 'count')]>=min_movies_rated]

top_rating_users = list(df_top_rating_users.index)

df_final=df_ratings_with_top_movies[df_ratings_with_top_movies['userId'].isin(top_rating_users)]

## **Item based collaborative filtering**

In [ ]:
df_user_item_matrix = df_final.pivot(index='movieId',columns='userId',values='rating').fillna(0)

In [ ]:
user_item_matrix_sparse = csr_matrix(df_user_item_matrix.values)

In [ ]:
print(user_item_matrix_sparse.shape)


(1060, 391)


In [ ]:
model = NearestNeighbors(n_neighbors=30, metric='cosine', algorithm='brute', n_jobs=-1)
model.fit(user_item_matrix_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=30)

# Movie Recommendation using K-Means

In [ ]:
import time

start = time.time()


index_to_movie = dict()

# Filter df_movies to include only rows present in df_user_item_matrix.index
df_titles_movies = df_movies[df_movies['movieId'].isin(df_user_item_matrix.index)].set_index('movieId')

count = 0

for index, row in df_titles_movies.iterrows():
    index_to_movie[count] = row['title']
    count += 1

def recommender(model, user_item_matrix_sparse, df_movies, number_of_recommendations, movie_index):
    try:
        main_title = index_to_movie[movie_index]
    except KeyError:
        print(f"Movie with movieId {movie_index} not found in the index.")
        return

    dist, ind = model.kneighbors(user_item_matrix_sparse[movie_index], n_neighbors=number_of_recommendations + 1)

    dist = dist[0].tolist()
    ind = ind[0].tolist()

    titles = []

    for index in ind:
        titles.append(index_to_movie.get(index, f"Unknown movie with index {index}"))

    recommendations = list(zip(titles, dist))

    # sort recommendations
    recommendations_sorted = sorted(recommendations, key=lambda x: x[1])

    # reverse recommendations, leaving out the first element
    recommendations_sorted.reverse()
    recommendations_sorted = recommendations_sorted[:-1]

    print("Recommendations for movie {}: ".format(main_title))

    count = 0

    for (title, distance) in recommendations_sorted:
        count += 1
        print('{}. {}, recommendation score = {}'.format(count, title, round(distance, 3)))

# Example usage
recommender(model, user_item_matrix_sparse, df_movies, 10, 1)


end = time.time()

print("\n------------------------------------------------")
print("Total Execution Time : {} Sec".format(end-start))
print("------------------------------------------------\n")

Recommendations for movie Jumanji (1995): 
1. Batman Forever (1995), recommendation score = 0.453
2. Toy Story (1995), recommendation score = 0.452
3. Casper (1995), recommendation score = 0.45
4. Mrs. Doubtfire (1993), recommendation score = 0.436
5. Twister (1996), recommendation score = 0.427
6. Jurassic Park (1993), recommendation score = 0.423
7. Aladdin (1992), recommendation score = 0.419
8. Mask, The (1994), recommendation score = 0.417
9. Lion King, The (1994), recommendation score = 0.399
10. Home Alone (1990), recommendation score = 0.379

------------------------------------------------
Total Execution Time : 0.0612335205078125 Sec
------------------------------------------------



In [ ]:
# index_to_movie = dict()

# df_titles_movies = df_movies.set_index('movieId').loc[df_user_item_matrix.index]

# count = 0

# for index, row in df_titles_movies.iterrows():

#     index_to_movie[count]=row['title']

#     count +=1


# def recommender(model, user_item_matrix_sparse, df_movies, number_of_recommendations, movie_index):

#     main_title = index_to_movie[movie_index]

#     dist, ind = model.kneighbors(user_item_matrix_sparse[movie_index], n_neighbors=number_of_recommendations+1)

#     dist = dist[0].tolist()

#     ind = ind[0].tolist()

#     titles = []

#     for index in ind:

#         titles.append(index_to_movie[index])

#     recommendations = list(zip(titles,dist))

#     # sort recommendations

#     recommendations_sorted = sorted(recommendations, key = lambda x:x[1])

#     # reverse recommendations, leaving out the first element

#     recommendations_sorted.reverse()

#     recommendations_sorted = recommendations_sorted[:-1]

#     print("Recommendations for movie {}: ".format(main_title))

#     count = 0

#     for (title, distance) in recommendations_sorted:

#         count += 1

#         print('{}. {}, recommendation score = {}'.format(count, title, round(distance,3)))

# recommender(model, user_item_matrix_sparse, df_movies, 10, 1)

# With SVD++

In [ ]:
df_SVD = df_final.sample(frac=0.01, random_state=42).reset_index(drop=True)

In [ ]:
minimum_rating = min(df_SVD['rating'].values)

maximum_rating = max(df_SVD['rating'].values)

print("Minimum/Maximum ratings: {}/{} ".format(minimum_rating,maximum_rating))

Minimum/Maximum ratings: 0.5/5.0 


In [ ]:
reader = Reader(rating_scale=(minimum_rating,maximum_rating))

data = Dataset.load_from_df(df_SVD[['userId', 'movieId', 'rating']], reader)

In [ ]:
algo = SVDpp()

algo.fit(data.build_full_trainset());

In [ ]:
user_id = '4'

movie_id = '10'

prediction = algo.predict(uid=user_id, iid=movie_id)

print("Predicted rating of user with id {} for movie with id {}: {}".format(user_id, movie_id, round(prediction.est,3)))

Predicted rating of user with id 4 for movie with id 10: 3.461


In [ ]:
import time

start = time.time()

def recommendations_from_SVDpp(user_id, df_user_item_matrix_SVD, algo, n_recommendations):

    # determine list of unrated movies by user_id

    list_of_unrated_movies = np.nonzero(df_user_item_matrix_SVD.loc[user_id].to_numpy() == 0)[0]

    # set up user set with unrated movies

    user_set = [[user_id, item_id, 0] for item_id in list_of_unrated_movies]


    # generate predictions

    predictions = algo.test(user_set)


    top_n_recommendations = defaultdict(list)

    for user_id, item_id, _, est, _ in predictions:

        top_n_recommendations[user_id].append((item_id, est))


    for user_id, ratings in top_n_recommendations.items():

        ratings.sort(key=lambda x: x[1], reverse=True)

        top_n_recommendations[user_id] = ratings[:n_recommendations]


    count = 0

    print("Recommendations for user with id {}: ".format(user_id))

    for movie_index, score in top_n_recommendations[user_id]:

        count +=1

        print('{}. {}, predicted rating = {}'.format(count, df_movies[df_movies['movieId']==movie_index]['title'].iloc[0], round(score,3)))

recommendations_from_SVDpp(4, df_user_item_matrix, algo, 10)

end = time.time()


print("\n------------------------------------------------")
print("Total Execution Time : {} Sec".format(end-start))
print("------------------------------------------------\n")

Recommendations for user with id 4: 
1. Godfather: Part II, The (1974), predicted rating = 4.192
2. Princess Bride, The (1987), predicted rating = 4.171
3. Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), predicted rating = 4.119
4. 2001: A Space Odyssey (1968), predicted rating = 4.101
5. Star Wars: Episode IV - A New Hope (1977), predicted rating = 4.097
6. Indiana Jones and the Last Crusade (1989), predicted rating = 4.057
7. Silence of the Lambs, The (1991), predicted rating = 4.054
8. Die Hard (1988), predicted rating = 4.054
9. Usual Suspects, The (1995), predicted rating = 4.046
10. Pulp Fiction (1994), predicted rating = 4.046

------------------------------------------------
Total Execution Time : 0.02000713348388672 Sec
------------------------------------------------



In [ ]:
from collections import defaultdict

def recommendations_from_SVDpp(user_id, df_user_item_matrix_SVD, algo, n_recommendations, output_file):

    # determine list of unrated movies by user_id
    list_of_unrated_movies = np.nonzero(df_user_item_matrix_SVD.loc[user_id].to_numpy() == 0)[0]

    # set up user set with unrated movies
    user_set = [[user_id, item_id, 0] for item_id in list_of_unrated_movies]

    # generate predictions
    predictions = algo.test(user_set)

    top_n_recommendations = defaultdict(list)

    for user_id, item_id, _, est, _ in predictions:
        top_n_recommendations[user_id].append((item_id, est))

    for user_id, ratings in top_n_recommendations.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        top_n_recommendations[user_id] = ratings[:n_recommendations]

    count = 0

    output_file.write("Recommendations for user with id {}: \n".format(user_id))

    for movie_index, score in top_n_recommendations[user_id]:
        count += 1
        movie_title = df_movies[df_movies['movieId']==movie_index]['title'].iloc[0]
        output_file.write('{}. {}, predicted rating = {}\n'.format(count, movie_title, round(score, 3)))

    print("Recommendations for user with id {}: ".format(user_id))

    for movie_index, score in top_n_recommendations[user_id]:
        count +=1
        print('{}. {}, predicted rating = {}'.format(count, df_movies[df_movies['movieId']==movie_index]['title'].iloc[0], round(score,3)))


# Open a file for writing
for x in []:
  with open('output_with_top_3_recomendations.txt', 'w') as file:
    recommendations_from_SVDpp(x, df_user_item_matrix, algo, 3, file)


In [ ]:
from collections import defaultdict
import numpy as np
import time

start = time.time()

def recommendations_from_SVDpp(user_id, df_user_item_matrix_SVD, algo, n_recommendations, output_file):

    # determine list of unrated movies by user_id
    list_of_unrated_movies = np.nonzero(df_user_item_matrix_SVD.loc[user_id].to_numpy() == 0)[0]

    # set up user set with unrated movies
    user_set = [[user_id, item_id, 0] for item_id in list_of_unrated_movies]

    # generate predictions
    predictions = algo.test(user_set)

    top_n_recommendations = defaultdict(list)

    for user_id, item_id, _, est, _ in predictions:
        top_n_recommendations[user_id].append((item_id, est))

    for user_id, ratings in top_n_recommendations.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        top_n_recommendations[user_id] = ratings[:n_recommendations]

    count = 0

    output_file.write("Recommendations for user with id {}: \n".format(user_id))

    for movie_index, score in top_n_recommendations[user_id]:
        count += 1
        movie_title = df_movies[df_movies['movieId'] == movie_index]['title'].iloc[0]
        output_file.write('{}. {}, predicted rating = {}\n'.format(count, movie_title, round(score, 3)))

    print("Recommendations for user with id {}: ".format(user_id))

    for movie_index, score in top_n_recommendations[user_id]:
        count += 1
        print('{}. {}, predicted rating = {}'.format(count, df_movies[df_movies['movieId'] == movie_index]['title'].iloc[0], round(score, 3)))

# Define your list of user IDs
user_ids = [1, 2, 3, 4]

# Open a file for writing
with open('output_with_top_3_recommendations.txt', 'w') as file:
    for user_id in user_ids:
        recommendations_from_SVDpp(user_id, df_user_item_matrix, algo, 3, file)


end = time.time()


print("\n------------------------------------------------")
print("Total Execution Time : {} Sec".format(end-start))
print("------------------------------------------------\n")

Recommendations for user with id 1: 
4. Nightmare Before Christmas, The (1993), predicted rating = 4.021
5. Chinatown (1974), predicted rating = 4.006
6. Forrest Gump (1994), predicted rating = 3.982
Recommendations for user with id 2: 
4. Princess Bride, The (1987), predicted rating = 4.171
5. Star Wars: Episode IV - A New Hope (1977), predicted rating = 4.097
6. Silence of the Lambs, The (1991), predicted rating = 4.054
Recommendations for user with id 3: 
4. Godfather: Part II, The (1974), predicted rating = 4.192
5. Princess Bride, The (1987), predicted rating = 4.171
6. Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), predicted rating = 4.119
Recommendations for user with id 4: 
4. Godfather: Part II, The (1974), predicted rating = 4.192
5. Princess Bride, The (1987), predicted rating = 4.171
6. Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), predicted rating = 4.119

------------------------------------------------
To